In [1]:
import numpy as np
import pandas as pd

In [2]:
# 定义获取显著性位点的函数
def get_sig_loci(df, p=1e-4):
    df = df[df['P'] < p]
    df = df.sort_values(by='P')
    df = df.reset_index(drop=True)
    return df

In [3]:
rice_fn_d = {}
f1 = open('rice_fn.txt', 'r')
for i in f1:
    if not i.startswith('Chr\t'):
        ls = i.strip().split('\t')
        chr = ls[0][3:]
        start = int(ls[1])
        end = int(ls[2])
        gene = ls[3]
        if chr not in rice_fn_d:
            rice_fn_d[chr] = {}
        rice_fn_d[chr][gene] = [start, end]
f1.close()

In [4]:
# 定义两个区域有重叠的函数
def overlap(a, b):
    return max(a[0], b[0]) < min(a[1], b[1])

In [7]:
th = 1/100000

In [5]:
# 定义位于显著性位点附近的基因的函数
def get_gene(df, dis = 150000,snp_num=3):
    # 遍历每个SNP的信息
    result = pd.DataFrame(columns=['SNP', 'CHR', 'GENE'])
    for i in range(df.shape[0]):
        # SNP是第i行snp列的值
        snp = df['SNP'][i]
        bp = df['BP'][i]
        chr = str(df['CHR'][i])
        bp_range = [bp-dis, bp+dis]
        for each_gene in rice_fn_d[chr]:
            if overlap(rice_fn_d[chr][each_gene], bp_range):
                result = result.append({'SNP':snp, 'CHR':chr, 'GENE':each_gene}, ignore_index=True)
        # 每个GENE至少被x个SNP覆盖
    result = result.groupby('GENE').filter(lambda x: x.shape[0] >= snp_num)
    return result

In [8]:
all_21_411 = pd.read_csv('./411_all_21y.txt', sep='\t', header=0)
all_21_411 = get_sig_loci(all_21_411, p=th)
print(all_21_411.shape)
all_21_411.head()

(418, 5)


,SNP,CHR,BP,P,BETA
0,rs12_22964137,12,22964137,4.811038e-09,13.812933
1,rs9_13991476,9,13991476,4.016004e-08,16.814210
2,rs3_11532929,3,11532929,4.258224e-08,21.391856
3,rs9_13990955,9,13990955,4.434320e-08,16.663180
4,rs1_29329055,1,29329055,4.947195e-08,11.706959


In [9]:
all_21_411_gene = get_gene(all_21_411, dis=150000, snp_num=3)
all_21_411_gene['P'] = all_21_411_gene['SNP'].apply(lambda x: all_21_411[all_21_411['SNP'] == x]['P'].values[0])
all_21_411_gene = all_21_411_gene.sort_values(by='P')
all_21_411_gene = all_21_411_gene.drop_duplicates(subset=['GENE'], keep='first')
print(all_21_411_gene.shape)
all_21_411_gene.head()

(558, 4)


,SNP,CHR,GENE,P
0,rs12_22964137,12,LOC_Os12g37200,4.811038e-09
26,rs12_22964137,12,LOC_Os12g37450,4.811038e-09
27,rs12_22964137,12,LOC_Os12g37460,4.811038e-09
28,rs12_22964137,12,LOC_Os12g37470,4.811038e-09
29,rs12_22964137,12,LOC_Os12g37480,4.811038e-09


In [11]:
xi_21 = pd.read_csv('./21y_xi_result.txt', sep='\t', header=0)
xi_21 = get_sig_loci(xi_21, p=th)
print(xi_21.shape)
xi_21.head()

(558, 5)


,SNP,CHR,BP,P,BETA
0,rs12_22964137,12,22964137,1.950522e-10,14.481087
1,rs9_13990955,9,13990955,1.694301e-08,16.374652
2,rs9_13991476,9,13991476,1.811468e-08,16.446136
3,rs12_22948668,12,22948668,2.587288e-08,12.177117
4,rs9_13991546,9,13991546,3.011421e-08,16.021217


In [12]:
xi_21_gene = get_gene(xi_21, dis=100000, snp_num=3)
xi_21_gene['P'] = xi_21_gene['SNP'].apply(lambda x: xi_21[xi_21['SNP'] == x]['P'].values[0])
xi_21_gene = xi_21_gene.sort_values(by='P')
xi_21_gene = xi_21_gene.drop_duplicates(subset=['GENE'], keep='first')
print(xi_21_gene.shape)
xi_21_gene.head()

(514, 4)


,SNP,CHR,GENE,P
0,rs12_22964137,12,LOC_Os12g37270,1.950522e-10
32,rs12_22964137,12,LOC_Os12g37570,1.950522e-10
31,rs12_22964137,12,LOC_Os12g37564,1.950522e-10
30,rs12_22964137,12,LOC_Os12g37560,1.950522e-10
29,rs12_22964137,12,LOC_Os12g37550,1.950522e-10


In [13]:
# 创建一个dataframe，用于存储所有的结果
result = pd.DataFrame(columns=['gene', 'phe', 'lead_snp', 'lead_snp_p'])
# 将所有的结果添加到result中
for i in all_21_411_gene['GENE']:
    result = result.append({'gene':i, 'phe':'all_21', 'lead_snp':all_21_411_gene[all_21_411_gene['GENE'] == i]['SNP'].values[0], 'lead_snp_p':all_21_411_gene[all_21_411_gene['GENE'] == i]['P'].values[0]}, ignore_index=True)
for i in xi_21_gene['GENE']:
    result = result.append({'gene':i, 'phe':'xi_21', 'lead_snp':xi_21_gene[xi_21_gene['GENE'] == i]['SNP'].values[0], 'lead_snp_p':xi_21_gene[xi_21_gene['GENE'] == i]['P'].values[0]}, ignore_index=True)
result.head()

,gene,phe,lead_snp,lead_snp_p
0,LOC_Os12g37200,all_21,rs12_22964137,4.811038e-09
1,LOC_Os12g37450,all_21,rs12_22964137,4.811038e-09
2,LOC_Os12g37460,all_21,rs12_22964137,4.811038e-09
3,LOC_Os12g37470,all_21,rs12_22964137,4.811038e-09
4,LOC_Os12g37480,all_21,rs12_22964137,4.811038e-09


In [14]:
# 按照gene进行排序
result = result.sort_values(by='gene')
result.head()

,gene,phe,lead_snp,lead_snp_p
389,LOC_Os01g07382,all_21,rs1_3636621,0.000003
256,LOC_Os01g07390,all_21,rs1_3646759,0.000001
257,LOC_Os01g07400,all_21,rs1_3646759,0.000001
258,LOC_Os01g07410,all_21,rs1_3646759,0.000001
265,LOC_Os01g07420,all_21,rs1_3646759,0.000001


In [15]:
# 保存result
result.to_csv('./gwas_gene_3SNP.result.txt', sep='\t', index=False, header=True,na_rep='NA')